# NGC 3568 Image Reducer

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter
3. Checking Drizzled images against the GAIA catalog

## Imports

In [ ]:
# Python Imports
import os
from os import path
from glob import iglob

# 3rd Party Imports
from matplotlib import pyplot as plt

# Astropy Colab Imports
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip
from astropy.table import QTable
from photutils.background import Background2D, SExtractorBackground as SEBack
from photutils.detection import IRAFStarFinder
from drizzlepac.tweakreg import TweakReg
from drizzlepac.astrodrizzle import AstroDrizzle

## Notebook Setup

In [ ]:
# DQ Bits
DQ_BAD_DET  = 4
DQ_HOT_PIX  = 16
DQ_CR_PIX   = 4096+8192
DQ_GOOD_PIX = ~(DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX)

## Functions

In [ ]:
# Write Sources to Catalog
def writecat(outName: str, srcs: QTable, wcs: WCS):
    
    # Get Coords
    crds = wcs.pixel_to_world(srcs['xcentroid'], srcs['ycentroid'])
    
    # Open File for Writing
    with open(outName, 'w') as fid:
        
        for crd in crds:
            
            fid.write(f'{crd.icrs.ra.value:20.10f} {crd.icrs.dec.value:20.10f}\n')

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in iglob('**/*flc.fits'):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

## Align Images to GAIA

### Align F275W to GAIA

The following method using TweakReg to find sources in the image kept returning garbage sources (essentially sources that were just background). I leave this code cell for posterity's sake (**don't run the cell below; it will fail**), but I will attempt another source finding method using PhotUtil in the next section.

In [ ]:
# # Image Find Parameters
# imagefindcfg = refimagefindcfg = dict(
#     peakmax=900,
#     threshold=40,
#     conv_width=3.5,
#     dqbits=DQ_GOOD_PIX
# )

# # Align the Images to the GAIA data
# TweakReg(
#     fileNameDict['F275W'],
#     updatehdr=False,
#     clean=False,
#     configobj=None,
#     refcat='../../../Data/GAIA/NGC3568-GAIA-RefCatalog.txt',
#     runfile='F275W-Tweak.log',
#     shiftfile=True,
#     outshifts='F275W-Shifts.log',
#     searchrad=1,
#     minobj=5,
#     imagefindcfg=imagefindcfg,
#     refimagefindcfg=refimagefindcfg
# )

#### F275W PhotUtil SourceID

Note that it would be more effecient to loop through all the files once. However, in terms of code readability and cell execution, I have broken up the image loader, background estimator, and source detector into a cell each.

In [ ]:
# Image Variables
imgs, dqs, wcss, msks = {}, {}, {}, {}

# Loop through Files
for fn in fileNameDict['F275W']:
    
    # Open the File
    with fits.open(fn) as hduList:
        
        # Get Images and DQs
        imgs[fn, 1] = hduList['sci', 1].data
        wcss[fn, 1] = WCS(hduList['sci', 1], hduList)
        imgs[fn, 2] = hduList['sci', 2].data
        wcss[fn, 2] = WCS(hduList['sci', 2], hduList)
        dqs[fn, 1]  = hduList['dq', 1].data
        dqs[fn, 2]  = hduList['dq', 2].data
    
    # Create the Masks
    msks[fn, 1] = ((dqs[fn, 1] & ~DQ_GOOD_PIX) != 0)
    msks[fn, 2] = ((dqs[fn, 2] & ~DQ_GOOD_PIX) != 0)

In [ ]:
# BG Vars
bgs = {}

# BG Estimator
# For changing default values
bkg_estimator = SEBack()
sigma_clip    = SigmaClip(maxiters=10)

# Get the Background for Each File
for fn in fileNameDict['F275W']:
    
    bgs[fn, 1] = Background2D(
        data=imgs[fn, 1],
        box_size=256,
        mask=msks[fn, 1],
        sigma_clip=sigma_clip,
        bkg_estimator=bkg_estimator
    )
    bgs[fn, 2] = Background2D(
        data=imgs[fn, 2],
        box_size=256,
        mask=msks[fn, 2],
        sigma_clip=sigma_clip,
        bkg_estimator=bkg_estimator
    )

In [ ]:
# Source Finder
irafFind = IRAFStarFinder(
    threshold=50,
    fwhm=3.5
)

# Find Sources and Write to File
catStr = ''
for fn in fileNameDict['F275W']:
    
    # Put the File Name in the catalog string
    catStr += path.abspath(fn)
    
    # Frame 1
    outName = path.join('tweak/F275W', path.basename(fn)[:-5] + '_1.coo')
    catStr += ' ' + outName
    srcs = irafFind(imgs[fn, 1] - bgs[fn, 1].background, mask=msks[fn, 1])
    writecat(outName, srcs, wcss[fn, 1])
    
    # Frame 2
    outName = path.join('tweak/F275W', path.basename(fn)[:-5] + '_2.coo')
    catStr += ' ' + outName + '\n'
    srcs = irafFind(imgs[fn, 2] - bgs[fn, 2].background, mask=msks[fn, 2])
    writecat(outName, srcs, wcss[fn, 2])

# Write the Catalog Name
with open('tweak/F275W/F275W_srcs.cat', 'w') as fid:
    fid.write(catStr)

#### Align F275W with IDed Sources

In [ ]:
# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F275W'],
    updatehdr=False,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    catfile='tweak/F275W/F275W_srcs.cat',
    xyunits='degrees',
    refcat='tweak/F275W/AlignmentStars-RefCat.txt',
    runfile='F275W-Tweak.log',
    shiftfile=True,
    outshifts='F275W-Shifts.log',
    searchrad=0.1,
    tolerance=2,
    minobj=5
)

In [ ]:
# Clean Up
for fn in iglob('*.match'):
    os.remove(fn)
for fn in iglob('*.coo'):
    os.remove(fn)
for fn in iglob('*.list'):
    os.remove(fn)
os.rename('F275W-Shifts.log', 'tweak/F275W/F275W-Shifts.log')
os.rename('F275W-Tweak.log', 'tweak/F275W/F275W-Tweak.log')
os.rename('shifts_wcs.fits', 'tweak/F275W/shift_wcs.fits')

### Align F475W to GAIA

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=40,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F475W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='tweak/F475W/AlignmentStars-RefCat.txt',
    runfile='F475W-Tweak.log',
    shiftfile=True,
    outshifts='F475W-Shifts.log',
    searchrad=0.1,
    minobj=15,
    tolerance=1,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg
)

In [ ]:
# Clean Up
for fn in iglob('*.match'):
    os.remove(fn)
for fn in iglob('*.coo'):
    os.remove(fn)
for fn in iglob('*.list'):
    os.remove(fn)
os.rename('F475W-Shifts.log', 'tweak/F475W/F475W-Shifts.log')
os.rename('F475W-Tweak.log', 'tweak/F475W/F475W-Tweak.log')
os.rename('shifts_wcs.fits', 'tweak/F475W/shift_wcs.fits')

### Align F814W to GAIA

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=40,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F814W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='tweak/F814W/AlignmentStars-RefCat.txt',
    runfile='F814W-Tweak.log',
    shiftfile=True,
    outshifts='F814W-Shifts.log',
    searchrad=0.1,
    minobj=15,
    tolerance=1,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg
)

In [ ]:
# Clean Up
for fn in iglob('*.match'):
    os.remove(fn)
for fn in iglob('*.coo'):
    os.remove(fn)
for fn in iglob('*.list'):
    os.remove(fn)
os.rename('F814W-Shifts.log', 'tweak/F814W/F814W-Shifts.log')
os.rename('F814W-Tweak.log', 'tweak/F814W/F814W-Tweak.log')
os.rename('shifts_wcs.fits', 'tweak/F814W/shift_wcs.fits')

## Drizzle Images

### Drizzle F814W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F814W'],
    output='F814W',
    runfile='F814W-Astro.log',
    wcskey='GAIA',
    build=True,
    restore=False,
    preserve=True,
    clean=True,
    final_wht_type='IVM',
    final_pixfrac=0.9,
    final_wcs=True,
    final_rot=0,
    final_scale=0.03,
    final_outnx=8500,
    final_outny=8500
)

### Drizzle F275W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F275W'],
    output='F275W',
    runfile='F275W-Astro.log',
    wcskey='GAIA',
    build=True,
    restore=False,
    preserve=True,
    clean=True,
    final_wht_type='IVM',
    final_pixfrac=0.90,
    final_wcs=True,
    final_refimage='F814W_drc.fits'
)

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F475W'],
    output='F475W',
    runfile='F475W-Astro.log',
    wcskey='GAIA',
    build=True,
    restore=False,
    preserve=True,
    clean=True,
    final_wht_type='IVM',
    final_pixfrac=0.90,
    final_wcs=True,
    final_refimage='F814W_drc.fits'
)